<a href="https://colab.research.google.com/github/mohamedSalehMatar/PRODIGY_ML_2/blob/main/PRODIGY_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [427]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [428]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sys

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


In [429]:
datatrain= pd.read_csv('/content/gdrive/MyDrive/InfoTech/TasksDone/Task2/Mall_Customers.csv')

In [430]:
datatrain.head(5)

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [431]:
datatrain.duplicated().sum()

0

In [432]:
#data null-value amout
datatrain.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

In [433]:
numerical_features = datatrain.select_dtypes(include=['float', 'int']).columns

for feature in numerical_features:
    #print(f'info of {feature} = ',datatrain[feature].describe())
    Q1 = np.percentile(datatrain[feature], 25, method='midpoint')
    Q3 = np.percentile(datatrain[feature], 75, method='midpoint')
    IQR = Q3 - Q1
    #print(f'\nIQR of {feature} = ', IQR)

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    #print(f'Lower bound of {feature} = ', lower)
    #print(f'Upper bound of {feature} = ', upper)

    # Replace the outliers with the feature mean
    datatrain.loc[(datatrain[feature] < lower) | (datatrain[feature] > upper), feature] = None

    # print(f'\nOutliers of {feature} is no more.')

In [434]:
datatrain.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        2
Spending Score (1-100)    0
dtype: int64

In [435]:
# Identify numerical columns
numerical_columns = datatrain.select_dtypes(['int', 'float']).columns

for column in numerical_columns:
    mean_value = datatrain[column].mean()  # Calculate the mean value
    datatrain[column].fillna(mean_value, inplace=True)  # Fill null values with the mean

# Verify the filled DataFrame
print(datatrain.isnull().sum())

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64


In [436]:
# Identify categorical columns
categorical_columns = datatrain.select_dtypes(['object']).columns

# One-hot encoding for each categorical column (using dummies)
for column in categorical_columns:
    encoded_cols = pd.get_dummies(datatrain[column], prefix=column, drop_first=True)
    datatrain = pd.concat([datatrain, encoded_cols], axis=1)
    datatrain.drop(column, axis=1, inplace=True)

In [437]:
datatrain.head(5)

,CustomerID,Age,Annual Income (k$),Spending Score (1-100),Gender_Male
0,1.0,19.0,15.0,39.0,True
1,2.0,21.0,15.0,81.0,True
2,3.0,20.0,16.0,6.0,False
3,4.0,23.0,16.0,77.0,False
4,5.0,31.0,17.0,40.0,False


In [438]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
datatrain_scaled = scaler.fit_transform(datatrain)

# Convert the transformed data back to a DataFrame
datatrain = pd.DataFrame(datatrain_scaled, columns=datatrain.columns)

In [439]:
datatrain.head(5)

,CustomerID,Age,Annual Income (k$),Spending Score (1-100),Gender_Male
0,0.000000,0.019231,0.000000,0.387755,1.0
1,0.005025,0.057692,0.000000,0.816327,1.0
2,0.010050,0.038462,0.009009,0.051020,0.0
3,0.015075,0.096154,0.009009,0.775510,0.0
4,0.020101,0.250000,0.018018,0.397959,0.0


In [440]:
#features
X = datatrain.iloc[:,:-1].values

#label
y = datatrain.iloc[:, -1].values

In [441]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

rows, cols = (20, 2)
arr = [[0]*cols for _ in range(rows)]  # Create an empty 2D array

for n in range(2, 22):
    # Create an instance of KMeans
    kmeans = KMeans(n_clusters=n, random_state=73, n_init=2)

    # Fit the KMeans model to the scaled data
    kmeans.fit(X)

    # Get the cluster labels for each data point
    labels = kmeans.labels_

    # Add the cluster labels to the DataFrame
    datatrain['Cluster'] = labels
    silhouette_avg = silhouette_score(X, labels)

    arr[n-2][0] = n  # Update the first element of the current row
    arr[n-2][1] = silhouette_avg  # Update the second element of the current row

print(arr)

[[2, 0.3215698813380069], [3, 0.3423351982639384], [4, 0.42015560294199517], [5, 0.40229696179559604], [6, 0.4281583871711612], [7, 0.42072270777804077], [8, 0.4099813587573148], [9, 0.3845653280286014], [10, 0.3670538986469027], [11, 0.37049999859956173], [12, 0.3568139291190263], [13, 0.35111345845582265], [14, 0.3417547535233743], [15, 0.31927966841837147], [16, 0.32177972490511], [17, 0.3253581809887098], [18, 0.3180470568653234], [19, 0.29842322039260344], [20, 0.2983251041113894], [21, 0.3456898665631874]]


In [442]:

# Find the row with the maximum silhouette_score
max_row_index = np.argmax(np.array(arr)[:, 1])
best_n = arr[max_row_index][0]
print(f"Best number of clusters: {best_n}")


Best number of clusters: 6
